In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

dat = pd.read_table("C:/Users/NTX550/Documents/카카오톡 받은 파일/info.csv",',')

In [3]:
dat = dat[dat['review'] != 'N']

dat.reset_index(drop=True,inplace=True)

dat = dat.astype({'score':'float32'})

dat['review_cnt'] = dat['review_cnt'].apply(lambda x:int(x.replace(',','')))

In [15]:
# ------------- review cosine_sim
count_vect_review = CountVectorizer(min_df=2, ngram_range=(1,2))

place_review = count_vect_review.fit_transform(dat['review'].astype('U'))

place_simi_review = cosine_similarity(place_review,place_review)
place_simi_review_sorted_ind = place_simi_review.argsort()[:,::-1]

# ------------- pos cosine_sim
count_vect_pos = CountVectorizer(min_df=2, ngram_range=(1,2))

place_pos = count_vect_pos.fit_transform(dat['pos'].astype('U'))

place_simi_pos = cosine_similarity(place_pos,place_pos)
place_simi_pos_sorted_ind = place_simi_pos.argsort()[:,::-1]

In [16]:
# voting
place_simi_co = (+ place_simi_pos * 0.3
                 + place_simi_review * 1 
                 + np.repeat([dat['review_cnt'].values], len(dat['review_cnt']) , axis=0) * 0.001  
                 + np.repeat([dat['score'].values], len(dat['score']) , axis=0) * 0.005)

place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 


# Recommendation Algorithm
def find_simi_place(dat, sorted_ind, place_name,place_pos, top_n=10):
    
    place_title = dat[(dat['상호명'] == place_name) & (dat['pos'] == place_pos)]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return dat.iloc[similar_indexes]



find_simi_place(dat, place_simi_co_sorted_ind, '스타벅스','강남역', 5)

,상호명,url,score,review_cnt,pos,review
194,스타벅스,https://m.place.naver.com/restaurant/1086067689,4.51,6112,강남역,강남역에 오면 매번 들르는 스타벅스매장~ 크리스마스캐롤 나오고 기분이 들떴어요~~~...
145,노브랜드,https://m.place.naver.com/restaurant/1740055509,4.44,6430,한티역,굿굿 친절해서 너무 좋아요 좋아요 노브랜드 바싹 군만두 기대중. 부디 먹을만하기를....
562,치즈룸,https://m.place.naver.com/restaurant/1297585187,4.58,6138,압구정역,살치살스테크 현미크리스피아이스크림 막걸리감자뇨끼 전복솥밥리조또 발사믹관자 스캄피알리...
420,어거스트힐,https://m.place.naver.com/restaurant/661051254,4.56,5560,신논현역,커플세트 먹었는데 양도 많고 맛있어요. 샐러드랑 스테이크 구성이라 살짝 배고플줄 알...
516,까페,https://m.place.naver.com/restaurant/1542149931,4.49,5505,선릉역,테이블 간격 넓어서 좋았어요 편하게 앉아서 친구들과 즐겁게 담소하고 왔네요 카페가 ...


In [31]:
# voting
place_simi_co = (+ place_simi_pos * 0.3
                 + place_simi_review * 1 
                 + np.repeat([dat['review_cnt'].values], len(dat['review_cnt']) , axis=0) * 0
                 + np.repeat([dat['score'].values], len(dat['score']) , axis=0) * 0.005)

place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 


# Recommendation Algorithm
def find_simi_place(dat, sorted_ind, place_name,place_pos, top_n=10):
    
    place_title = dat[(dat['상호명'] == place_name) & (dat['pos'] == place_pos)]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return dat.iloc[similar_indexes]



find_simi_place(dat, place_simi_co_sorted_ind, '스타벅스','강남역', 5)

,상호명,url,score,review_cnt,pos,review
194,스타벅스,https://m.place.naver.com/restaurant/1086067689,4.51,6112,강남역,강남역에 오면 매번 들르는 스타벅스매장~ 크리스마스캐롤 나오고 기분이 들떴어요~~~...
102,할리스커피,https://m.place.naver.com/restaurant/1052747813,4.43,2880,강남역,"매장이 1층 지하층으로 이루어져있고, 엄청 넓어요~ 공부하시는 분들도 많고 이야기 ..."
577,슬링,https://m.place.naver.com/restaurant/36893827,4.44,267,강남역,"라자냐, 오믈렛 진짜 맛있어요! 오믈렛 저 소스가 꾸덕하니 미쳤는데 올려진 다진고기..."
406,라이브,https://m.place.naver.com/restaurant/1801857894,4.35,154,강남역,칵테일 종류가 진짜 많아요! 그리고 엄청 맛있어요… 술이 아니라 음료 같이 너무 맛...
680,브라운홀릭,https://m.place.naver.com/restaurant/20561475,4.45,675,강남역,소금커피 : 짜고 크리미 아메리카노 가격에 비해 커피가 굿 맛있다. 정말 감동 싸다...
